In [1]:
# load python modules
import Bio
from Bio import Entrez
from Bio import SeqIO
from dicttoxml import dicttoxml


## Part 1
1. (esearch) queries the base of nucleotide sequences for all sequences according to the name of the gene ... (for example, GSPT1) for the organism ... (for example, human) and returns xml;	

In [3]:
Entrez.email = 'anton.chana@gmail.com' # provide your email address
handle = Entrez.esearch(db="nucleotide", term='"human"[Organism] AND GSPT1[Gene]', rettype = 'xml')

In [4]:
# handle - xml
# get eSearch result as dict object
record = Entrez.read(handle)
print(record)

{'Count': '9', 'RetMax': '9', 'RetStart': '0', 'IdList': ['1676355513', '1676319656', '1519312966', '568815582', '74273666', '74230029', '33874733', '39754980', '307685420'], 'TranslationSet': [{'From': '"human"[Organism]', 'To': '"Homo sapiens"[Organism]'}], 'TranslationStack': [{'Term': '"Homo sapiens"[Organism]', 'Field': 'Organism', 'Count': '27614143', 'Explode': 'Y'}, {'Term': 'GSPT1[Gene]', 'Field': 'Gene', 'Count': '1020', 'Explode': 'N'}, 'AND'], 'QueryTranslation': '"Homo sapiens"[Organism] AND GSPT1[Gene]'}


2. (for example, esearch + esummary) returns a table with UID (in XML this field is called Id), accession number (in XML this field is called Caption), sequence length;


In [5]:
handle = Entrez.esummary(db="nucleotide", id="1676355513")
record = Entrez.read(handle)
print(record)
print("Protein info\nid: {}\nAccession number: {}\nLength: {}".format (record[0]["Caption"], record[0]["Id"], record[0]["Length"]))


[{'Item': [], 'Id': '1676355513', 'Caption': 'NM_001130006', 'Title': 'Homo sapiens G1 to S phase transition 1 (GSPT1), transcript variant 2, mRNA', 'Extra': 'gi|1676355513|ref|NM_001130006.2|[1676355513]', 'Gi': IntegerElement(1676355513, attributes={}), 'CreateDate': '2008/07/11', 'UpdateDate': '2020/03/01', 'Flags': IntegerElement(512, attributes={}), 'TaxId': IntegerElement(9606, attributes={}), 'Length': IntegerElement(7138, attributes={}), 'Status': 'live', 'ReplacedBy': '', 'Comment': '  ', 'AccessionVersion': 'NM_001130006.2'}]
Protein info
id: NM_001130006
Accession number: 1676355513
Length: 7138


3. (for example, esearch + efetch) returns the nucleotide sequences in fasta format and writes to the file;*

In [6]:
handle = Entrez.efetch(db="nucleotide", id="EU490707", rettype="fasta", retmode="text")
with open('nucl.fasta', 'w') as f:
    f.write(handle.read())
#and check:
with open('nucl.fasta', 'r') as f:
    print(f.read())


>EU490707.1 Selenipedium aequinoctiale maturase K (matK) gene, partial cds; chloroplast
ATTTTTTACGAACCTGTGGAAATTTTTGGTTATGACAATAAATCTAGTTTAGTACTTGTGAAACGTTTAA
TTACTCGAATGTATCAACAGAATTTTTTGATTTCTTCGGTTAATGATTCTAACCAAAAAGGATTTTGGGG
GCACAAGCATTTTTTTTCTTCTCATTTTTCTTCTCAAATGGTATCAGAAGGTTTTGGAGTCATTCTGGAA
ATTCCATTCTCGTCGCAATTAGTATCTTCTCTTGAAGAAAAAAAAATACCAAAATATCAGAATTTACGAT
CTATTCATTCAATATTTCCCTTTTTAGAAGACAAATTTTTACATTTGAATTATGTGTCAGATCTACTAAT
ACCCCATCCCATCCATCTGGAAATCTTGGTTCAAATCCTTCAATGCCGGATCAAGGATGTTCCTTCTTTG
CATTTATTGCGATTGCTTTTCCACGAATATCATAATTTGAATAGTCTCATTACTTCAAAGAAATTCATTT
ACGCCTTTTCAAAAAGAAAGAAAAGATTCCTTTGGTTACTATATAATTCTTATGTATATGAATGCGAATA
TCTATTCCAGTTTCTTCGTAAACAGTCTTCTTATTTACGATCAACATCTTCTGGAGTCTTTCTTGAGCGA
ACACATTTATATGTAAAAATAGAACATCTTCTAGTAGTGTGTTGTAATTCTTTTCAGAGGATCCTATGCT
TTCTCAAGGATCCTTTCATGCATTATGTTCGATATCAAGGAAAAGCAATTCTGGCTTCAAAGGGAACTCT
TATTCTGATGAAGAAATGGAAATTTCATCTTGTGAATTTTTGGCAATCTTATTTTCACTTTTGGTCTCAA
CCGTATAGGATTCATATAAAGCAATTATCCAACTATTCCTTCTCTTTTCTGGGGTATTTT

4. (elink + efetch) downloads all sequences from the paper with given PMID ... (for example, 12890024)

In [6]:
with open('paper_data.fasta', 'w') as output:
    handle = Entrez.elink(dbfrom="pubmed", id='12890024', linkname="pubmed_pubmed")
    record = Entrez.read(handle)
    for index in record[0]['LinkSetDb'][0]['Link']:
        try:
            handle = Entrez.efetch(db="nucleotide", id=index['Id'], rettype="fasta", retmode="text")
            output.write(handle.read())
        except:
            continue

In [11]:
#check (проверим, что чтото скачалось!):
with open('paper_data.fasta', 'r') as input_f:
    print(input_f.read(1000))

>AZ769660.1 1M0570P06R Mouse 10kb plasmid UUGC1M library Mus musculus genomic clone UUGC1M0570P06 R, genomic survey sequence
TCTGGCTCGTTCCTCTGAAAACAAGGATTGCACAGAGTCATTTTTAAAGAATCTATTCATTTTTGAATTT
TCCCTCCAATAACACCTTCAGTTCTCTCTGTACCATTTCCCACAGNAGGAAGAAAATAGTATGTATTTGT
CCCATTCTTCTGTGCTGTGCTCATGTGCTATGAACATGTGTGCACATACATGTGGAGGTGTCAGGACTCA
GCCTCCGCCACTCTTCTAGCTTATTTAGTGAGGCAGGGTCTTCCTGCAAACCCTAGAGCTCACCAATACA
GCTCGTCTTGCCAGCCAGCTTGCTCTGGGAATTTCCTGTCTCTGCCTTC

>AW692118.2 NF047H03ST1F1000 Developing stem Medicago truncatula cDNA clone NF047H03ST 5', mRNA sequence
ATTACATCCCATAGATTCCCAACACTTTTTAAACCCTAGCGGCACTACTTTACAGTTCAAACCCTTTCTC
TCTCTAGAACATGGCCGATCCCGAGCACCGCGAAGAGGAGGAAGCACCCGCCGTCGGTGACGACGAAGAC
ACCGGAGCACAAGTCGCTCCGATTGTCCAACTCCAAGAAGTTGCCGTCACTACCGGTGAAGAAGACGAAG
AATCCATTCTAGATCTCAAATCAAAGCTTTACCGTTTCGATAAAGATGGAAATCAGTGGAAGGAGAGAGG
TGCTGGGA

>AZ510847.1 1M0355P01R Mouse 10kb plasmid UUGC1M library Mus musculus genomic clone UUGC1M0355P01 R, genomic survey sequence
TCCAACCATGGTTTCT

## Part 2
data: https://drive.google.com/drive/folders/1d1m_fBZ8YrBydNwsvSupHSZ2MORIiFSV
1. What sequences were used for analysis? Provide type of biopolymer, organism, gene name.

In [52]:
!cat seqs/*

>SUP35_Kla_AB039749
ATGTCAGACCAACAAAATCAAGACCAAGGGCAAGGCCAAGGTTACAATCAGTATAACCAATATGGCCAGT
ACAACCAGTACTACAACCAACAGGGCTATCAAGGCTACAACGGCCAACAAGGTGCTCCTCAAGGCTACCA
AGCATATCAAGCTTATGGACAGCAGCCTCAAGGAGCCTACCAGGGCTACAACCCTCAACAAGCTCAAGGC
TACCAACCTTACCAGGGCTACAATGCTCAGCAACAAGGTTACAACGCTCAGCAAGGCGGTCACAACAATA
ACTACAATAAAAATTATAATAATAAAAACAGTTACAATAACTATAATAAGCAGGGTTATCAAGGTGCTCA
AGGATATAATGCACAACAGCCAACCGGTTACGCTGCTCCAGCACAGTCTTCATCCCAGGGTATGACTTTG
AAAGATTTCCAAAACCAACAAGGCAGTACTAATGCAGCCAAGCCAAAGCCTAAGTTAAAGTTGGCCTCTA
GCTCTGGTATTAAGTTAGTAGGTGCCAAGAAACCTGTAGCACCCAAAACTGAGAAAACTGATGAATCCAA
GGAAGCAACTAAAACTACCGACGACAACGAAGAAGCACAATCTGAATTGCCCAAAATTGATGATTTGAAA
ATCTCAGAGGCTGAAAAACCAAAAACTAAGGAGAATACCCCATCTGCTGATGATACTTCCTCAGAGAAGA
CTACCAGCGCTAAGGCAGACACATCTACAGGAGGAGCGAACTCCGTGGATGCTCTAATCAAGGAACAAGA
AGATGAGGTTGACGAAGAAGTCGTTAAAGATATGTTTGGTGGTAAGGATCATGTTTCCATCATTTTCATG
GGTCACGTCGATGCTGGTAAGTCAACAATGGGTGGTAACTTGTTATATCTGACTGGTTCTGTGGATAAAA
GAACCGTTGAGAAATATGAGAGAGAGGCTAAAGAGGCTGGTAGACAAGGTTGGTATT

AAAGGTCGAAGAACCAGTTAAAAAGGAGGAGAAACCAGTCCAGACTGAAGAAAAGAAGGAGGAAAAATCG
GAACTTCCAAAGGTAGAAGACCTCAAAATCTCTGAATCAACAGATAATACCAACAATGCCAATGTTACCA
GTGCTGATGCCTTGATCAAGGAACAGGAAGAAGAAGTGGATGACGAAGTTGTTAACGATATGTTTGGTGG
TAAAGATCACGTTTCTTTAATTTTCATGGGTCATGTTGATGCCGGTAAATCTACTATGGGTGGTAATCTA
CTATACTTGACTGGCTCTGTGGATAAGAGAACTATTGAGAAATATGAAAGAGAAGCCAAGGATGCAGGCA
GACAAGGTTGGTACTTGTCATGGGTCATGGATACCAACAAAGAAGAAAGAAATGATGGTAAGACTATCGA
AGTTGGTAAGGCCTACTTTGAAACTGAAAAAAGGCGTTATACCATATTGGATGCTCCTGGTCATAAAATG
TACGTTTCCGAGATGATCGGTGGTGCTTCTCAAGCTGATGTTGGTGTTTTGGTCATTTCCGCCAGAAAGG
GTGAGTACGAAACCGGTTTTGAGAGAGGTGGTCAAACTCGTGAACATGCCCTATTGGCCAAGACCCAAGG
TGTTAATAAGATGGTTGTCGTCGTAAATAAGATGGATGACCCAACCGTTAACTGGTCTAAGGAACGTTAC
GACCAATGTGTGAGTAATGTCAGCAATTTCTTGAGAGCAATTGGTTACAACATTAAGACAGACGTTGTAT
TTATGCCAGTATCCGGTTACAGTGGTGCAAATTTGAAAGATCACGTAGATCCAAAAGAATGCCCATGGTA
CACCGGCCCAACTCTGTTAGAATATCTGGATACAATGAACCACGTCGACCGTCACATCAATGCTCCATTC
ATGTTGCCTATTGCCGCTAAGATGAAGGATCTAGGTACCATCGTTGAAGGTAAAATTGAATCCGGTCATA
TCAAAA

Были спользованы молекулы ДНК. (нуклеотиды)

In [53]:
!cat seqs/SUP35_10seqs.fa | grep -e '>'

>SUP35_Kla_AB039749
>SUP35_Agos_ATCC_10895_NM_211584
>SUP35_Scer_74-D694_GCA_001578265.1
>SUP35_Spar_A12_Liti
>SUP35_Smik_IFO1815T_30
>SUP35_Skud_IFO1802T_36
>SUP35_Sbou_unique28_CM003560
>SUP35_Scer_beer078_CM005938
>SUP35_Sarb_H-6_chrXIII_CM001575
>SUP35_Seub_CBS12357_chr_II_IV_DF968535


Поищу в ncbi по нуклеотидам
>SUP35_Kla_AB039749  - Kluyveromyces lactis sup35 gene for polypeptide release factor 3, complete cds.

>SUP35_Agos_ATCC_10895_NM_211584 - Eremothecium gossypii ATCC 10895 AGL145Wp (AGOS_AGL145W), partial mRNA

>SUP35_Scer_74-D694_GCA_001578265.1 - Saccharomyces cerevisiae strain, Torulaspora delbrueckii gene

>SUP35_Spar_A12_Liti - Rpa12 [Saccharomyces paradoxus]

>SUP35_Smik_IFO1815T_30 - S. mikatae IFO1815 - S. mikatae strain IFO1815 complex of fermentation

>SUP35_Skud_IFO1802T_36 - S. kudriavzevii

>SUP35_Sbou_unique28_CM003560 - Saccharomyces sp. 'boulardii' strain unique28 chromosome IV, whole genome shotgun sequence (кусок)

>SUP35_Scer_beer078_CM005938 - Saccharomyces cerevisiae strain beer078 chromosome IV, whole genome shotgun sequence (кусок)

>SUP35_Sarb_H-6_chrXIII_CM001575 - Saccharomyces arboricola H-6 chromosome XIII, whole genome shotgun sequence (кусок)

>SUP35_Seub_CBS12357_chr_II_IV_DF968535 - Saccharomyces eubayanus DNA, scaffold: CBS12357_chr_II_IV, strain: CBS12357, whole genome shotgun sequence (кусок)

 


2. Run at least 4 different alignment algorithms (clustalw, muscle, mafft, kalign, tcoffee, prank) for 10 DNA sequences (SUP35_10seqs.fa) + parameters if applicable. Create comparison table with running time* and comments on the DNA alignment quality for the above algorithms**. Which algorithm is better to use?


In [9]:
!time clustalw seqs/SUP35_10seqs.fa




 CLUSTAL 2.1 Multiple Sequence Alignments


Sequence format is Pearson
Sequence 1: SUP35_Kla_AB039749                      2103 bp
Sequence 2: SUP35_Agos_ATCC_10895_NM_211584         2076 bp
Sequence 3: SUP35_Scer_74-D694_GCA_001578265.1      2058 bp
Sequence 4: SUP35_Spar_A12_Liti                     2046 bp
Sequence 5: SUP35_Smik_IFO1815T_30                  2046 bp
Sequence 6: SUP35_Skud_IFO1802T_36                  2037 bp
Sequence 7: SUP35_Sbou_unique28_CM003560            2058 bp
Sequence 8: SUP35_Scer_beer078_CM005938             2001 bp
Sequence 9: SUP35_Sarb_H-6_chrXIII_CM001575         2043 bp
Sequence 10: SUP35_Seub_CBS12357_chr_II_IV_DF968535  2025 bp
Start of Pairwise alignments
Aligning...

Sequences (1:2) Aligned. Score:  70
Sequences (1:3) Aligned. Score:  71
Sequences (1:4) Aligned. Score:  71
Sequences (1:5) Aligned. Score:  71
Sequences (1:6) Aligned. Score:  71
Sequences (1:7) Aligned. Score:  71
Sequences (1:8) Aligned. Score:  69
Sequences (1:9) Aligned. Score:

In [10]:
!mv seqs/SUP35_10seqs.aln seqs/clustalw_SUP35_10seqs.aln

Сконвертируем в фасту:

In [21]:
from Bio import SeqIO

records = SeqIO.parse("seqs/clustalw_SUP35_10seqs.aln", "clustal")
count = SeqIO.write(records, "seqs/clustalw_SUP35_10seqs.fasta", "fasta")
print("Converted %i records" % count)

Converted 10 records


In [12]:
!time muscle -in seqs/SUP35_10seqs.fa -out seqs/muscle_SUP35_10seqs.fa


MUSCLE v3.8.31 by Robert C. Edgar

http://www.drive5.com/muscle
This software is donated to the public domain.
Please cite: Edgar, R.C. Nucleic Acids Res 32(5), 1792-97.

SUP35_10seqs 10 seqs, max length 2103, avg  length 2049
00:00:00    24 MB(-4%)  Iter   1  100.00%  K-mer dist pass 1
00:00:00    24 MB(-4%)  Iter   1  100.00%  K-mer dist pass 2
00:00:01    45 MB(-7%)  Iter   1  100.00%  Align node       
00:00:01    45 MB(-7%)  Iter   1  100.00%  Root alignment
00:00:01    45 MB(-7%)  Iter   2  100.00%  Refine tree   
00:00:01    45 MB(-7%)  Iter   2  100.00%  Root alignment
00:00:01    45 MB(-7%)  Iter   2  100.00%  Root alignment
00:00:03    45 MB(-7%)  Iter   3  100.00%  Refine biparts
00:00:04    45 MB(-7%)  Iter   4  100.00%  Refine biparts
00:00:04    45 MB(-7%)  Iter   5  100.00%  Refine biparts
00:00:04    45 MB(-7%)  Iter   5  100.00%  Refine biparts
3.62user 0.02system 0:03.65elapsed 99%CPU (0avgtext+0avgdata 24796maxresident)k
0inputs+48outputs (0major+6705minor)pagefault

In [16]:
!time mafft --auto seqs/SUP35_10seqs.fa > seqs/mafft_SUP35_10seqs.fa


generating a scoring matrix for nucleotide (dist=200) ... done
All-to-all alignment.
tbfast-pair (nuc) Version 7.310 alg=L, model=DNA200 (2), 2.00 (6.00), -0.10 (-0.30), noshift, amax=0.0
0 thread(s)

Loading 'hat3.seed' ... 
done.
Writing hat3 for iterative refinement
generating a scoring matrix for nucleotide (dist=200) ... done
Gap Penalty = -1.53, +0.00, +0.00
treein = 0
compacttree = 0
Constructing a UPGMA tree ... 
    0 / 10
done.

Progressive alignment ... 
STEP     8 /9 c
Reallocating..done. *alloclen = 5227
STEP     9 /9 c
done.
tbfast (nuc) Version 7.310 alg=A, model=DNA200 (2), 1.53 (4.59), -0.00 (-0.00), noshift, amax=0.0
0 thread(s)

minimumweight = 0.000010
autosubalignment = 0.000000
nthread = 0
randomseed = 0
blosum 62 / kimura 200
poffset = 0
niter = 16
sueff_global = 0.100000
Loading 'hat3' ... done.
generating a scoring matrix for nucleotide (dist=200) ... done

    0 / 10
Segment   1/  1    1-2219
STEP 003-003-1  rejected..    identical.    identical.    identical

In [18]:
!time kalign seqs/SUP35_10seqs.fa seqs/kalign_SUP35_10seqs.fa

EXTRA :2

Kalign version 2.04, Copyright (C) 2004, 2005, 2006 Timo Lassmann

        Kalign is free software. You can redistribute it and/or modify
        it under the terms of the GNU General Public License as
        published by the Free Software Foundation.

reading from STDIN: found no sequences.

reading from seqs/SUP35_10seqs.fa: found 10 sequences
reading from seqs/kalign_SUP35_10seqs.fa: found 10 sequences
Aligning 20 RNA/DNA sequences with these parameters:
        217.00000000	gap open penalty
        39.40000153	gap extension
        292.60000610	terminal gap penalty
        283.00000000	bonus
Alignment will be written to stdout.

Distance Calculation:
     100 percent done
Alignment:
       0 percent doneSaving mem...
       5 percent doneSaving mem...
      10 percent doneSaving mem...
      15 percent doneSaving mem...
      20 percent doneSaving mem...
      25 percent doneSaving mem...
      30 percent doneSaving mem...
      35 percent doneSaving mem...
      40 perc

In [19]:
!time prank -d=seqs/SUP35_10seqs.fa -o=seqs/prank_SUP35_10seqs.fa


-----------------
 PRANK v.170427:
-----------------

Input for the analysis
 - aligning sequences in 'seqs/SUP35_10seqs.fa'
 - using inferred alignment guide tree
 - option '+F' is not used; it can be enabled with '+F'
 - external tools available:
    MAFFT for initial alignment

Correcting (arbitrarily) for multifurcating nodes.
Correcting (arbitrarily) for multifurcating nodes.

Generating multiple alignment: iteration 1.

Alignment score: 4527
Correcting (arbitrarily) for multifurcating nodes.

Generating multiple alignment: iteration 2.

Alignment score: 4514
Correcting (arbitrarily) for multifurcating nodes.

Generating multiple alignment: iteration 3.

Alignment score: 4514
Correcting (arbitrarily) for multifurcating nodes.

Generating multiple alignment: iteration 4.

Alignment score: 4514
Correcting (arbitrarily) for multifurcating nodes.

Generating multiple alignment: iteration 5.

Alignment score: 4514


Writing
 - alignment to 'seqs/kalign_SUP35_10seqs.fa.best.fas'

Analy

Посмотрим на сводку алайнеров от Полины:
![](aligners.png)

Для оценки запихнем в [gblocks](http://molevol.cmima.csic.es/castresana/Gblocks_server.html)

Сравним:
* clustalw
    * Время: 5.14 сек
    * New number of positions in clustalw_SUP35_10seqs.fasta-gb:  1861  (86% of the original 2148 positions)
    * т.е. скор 86%
* muscle
    * Время: 3.62 сек
    * New number of positions in muscle_SUP35_10seqs.fa-gb:  1753  (77% of the original 2275 positions)
    * т.е. скор 77%
* mafft
    * Время: 3.66 сек
    * New number of positions in mafft_SUP35_10seqs.fa-gb:  1870  (86% of the original 2166 positions)
    * т.е. скор 86%
* kalign
    * Время: 0.92 сек
    * New number of positions in kalign_SUP35_10seqs.fa-gb:  1886  (87% of the original 2155 positions)
    * т.е. скор 87%
* prank
    * Время: 99.8 сек (но он крутой, знает филогенетику)
    * New number of positions in prank_SUP35_10seqs.fa.best.fas-gb:  1696  (71% of the original 2366 positions)
    * т.е скор 71%
 
Интересно, пранк, знающий филогенетику показал самый низкий скор (а работал дольше всех)). Может, потому что плохо выравниваются (?). Самое интересное, что сначала у него перед итерациями: MAFFT for initial alignment, у которого самый высокий скор. 0_0

3. What is wrong with the alignment of SUP35_10seqs_strange_aln.fa and how to fix it?

In [22]:
!cat seqs/SUP35_10seqs_strange_aln.fa

>SUP35_Kla_AB039749
ATGTCAG---------------------------ACCAACAAAATCAAGACCAAGGGCAAGGCCAAGGTTACAATCAGTATAACCAATATGGCCAGTACAACCAG----TACTACAACCAACAGGGCTATCAAGGCTACAACGGCCAACAAGGTGCTCCTCAAGGCTACCAAGCATATCAAGCTTATGGACAGCAGCCTCAAGG---------------------AGCCTACCAGGGCTAC---AACCCTC--AACAAGCTCAAG-----GCT---ACCAACCTTACCAGGGCT---ACAATGCTCAGCAA------CAAGGTTACAAC-----------GCTCAG------------------------CAAGGCGGTCA-CAACAATAACTACAATAAAAA--------TTATAATAATAA---AAACAGTTACAATAACTATAATAAGCA-GGGTTATCAAGGTGCTCAAGGATATAATGCACAACAGCCAACCGGTTACGCTGCTCCAGCACAGTCTTCATCCCAGGGTATGACT-----TTGAAAGATTTCCAAAA-------CCAAC-AAGGCAGTACTAATGCAGCCAAGCCAAAGCCTAAGTTAAAGTTGGCCTCTAGCTCTGGTATTAAGTTAGTAGGTG---CCAAGAAACCTGTAGCA------CCCAAAACTGAGAAAA-----------------CTGATGAATC--CAAGGAAGCAACTAAA------------------------------------------------ACTACCGACGACAACGAAGAAGCACAATCTGAATTGCCCAAAATTGATG-ATTTGAAAATCTCAGAGGCTGAAAAACCAAAAACTAAGGAGAATACCCCATCTGCTGATGATACTTCCTCAGAGAAGACTACCAGCGCTAAGGCAGACACATCTACAGGAGGAGCGAACTCCGTGGAT------GCTCTAATCAAG

В этом файле все, кроме одного рида (>SUP35_Spar_A12_Liti_) начинаются на ATG - старт кодон, кодирующий метионин.

Как исправить? Здесь много ATG в последовательности, возможно стоит обрезать до них и выровнять. (учитывая, что там ольшие пропуски в начале


4. Obtain amino acid sequences for these data. Repeat p.2 on amino acid sequences.

Перевел вот тут: https://www.ebi.ac.uk/Tools/services/web/toolresult.ebi?jobId=emboss_transeq-I20200320-120413-0070-90224078-p2m

и сохранил как protein.fa

In [26]:
!time clustalw seqs/protein.fa
!mv seqs/protein.aln seqs/clustal_protein.aln




 CLUSTAL 2.1 Multiple Sequence Alignments


Sequence format is Pearson
Sequence 1: SUP35_Kla_AB039749_1                       700 aa
Sequence 2: SUP35_Agos_ATCC_10895_NM_211584_1          691 aa
Sequence 3: SUP35_Scer_74-D694_GCA_001578265.1_1       685 aa
Sequence 4: SUP35_Spar_A12_Liti_1                      681 aa
Sequence 5: SUP35_Smik_IFO1815T_30_1                   681 aa
Sequence 6: SUP35_Skud_IFO1802T_36_1                   678 aa
Sequence 7: SUP35_Sbou_unique28_CM003560_1             685 aa
Sequence 8: SUP35_Scer_beer078_CM005938_1              666 aa
Sequence 9: SUP35_Sarb_H-6_chrXIII_CM001575_1          680 aa
Sequence 10: SUP35_Seub_CBS12357_chr_II_IV_DF968535_1   674 aa
Start of Pairwise alignments
Aligning...

Sequences (1:2) Aligned. Score:  71
Sequences (1:3) Aligned. Score:  69
Sequences (1:4) Aligned. Score:  69
Sequences (1:5) Aligned. Score:  68
Sequences (1:6) Aligned. Score:  68
Sequences (1:7) Aligned. Score:  69
Sequences (1:8) Aligned. Score:  70
Sequences (

In [27]:
from Bio import SeqIO
records = SeqIO.parse("seqs/clustal_protein.aln", "clustal")
count = SeqIO.write(records, "seqs/clustal_protein.fasta", "fasta")
print("Converted %i records" % count)

Converted 10 records


In [28]:
!time muscle -in seqs/protein.fa -out seqs/muscle_protein.fa


MUSCLE v3.8.31 by Robert C. Edgar

http://www.drive5.com/muscle
This software is donated to the public domain.
Please cite: Edgar, R.C. Nucleic Acids Res 32(5), 1792-97.

protein 10 seqs, max length 700, avg  length 682
00:00:00    23 MB(-4%)  Iter   1  100.00%  K-mer dist pass 1
00:00:00    23 MB(-4%)  Iter   1  100.00%  K-mer dist pass 2
00:00:00    30 MB(-5%)  Iter   1  100.00%  Align node       
00:00:00    30 MB(-5%)  Iter   1  100.00%  Root alignment
00:00:00    30 MB(-5%)  Iter   2  100.00%  Refine tree   
00:00:00    30 MB(-5%)  Iter   2  100.00%  Root alignment
00:00:00    30 MB(-5%)  Iter   2  100.00%  Root alignment
00:00:00    30 MB(-5%)  Iter   3  100.00%  Refine biparts
00:00:00    30 MB(-5%)  Iter   4  100.00%  Refine biparts
00:00:00    30 MB(-5%)  Iter   5  100.00%  Refine biparts
00:00:00    30 MB(-5%)  Iter   5  100.00%  Refine biparts
00:00:00    30 MB(-5%)  Iter   6  100.00%  Refine biparts
00:00:00    30 MB(-5%)  Iter   7  100.00%  Refine biparts
00:00:00    30 M

In [29]:
!time mafft --auto seqs/protein.fa > seqs/mafft_protein.fa


All-to-all alignment.
tbfast-pair (aa) Version 7.310 alg=L, model=BLOSUM62, 2.00, -0.10, +0.10, noshift, amax=0.0
0 thread(s)

Loading 'hat3.seed' ... 
done.
Writing hat3 for iterative refinement
Gap Penalty = -1.53, +0.00, +0.00
treein = 0
compacttree = 0
Constructing a UPGMA tree ... 
    0 / 10
done.

Progressive alignment ... 
STEP     8 /9 c
Reallocating..done. *alloclen = 2413
STEP     9 /9 c
done.
tbfast (aa) Version 7.310 alg=A, model=BLOSUM62, 1.53, -0.00, -0.00, noshift, amax=0.0
0 thread(s)

minimumweight = 0.000010
autosubalignment = 0.000000
nthread = 0
randomseed = 0
blosum 62 / kimura 200
poffset = 0
niter = 16
sueff_global = 0.100000
Loading 'hat3' ... done.

    0 / 10
Segment   1/  1    1- 753
STEP 003-003-1  identical.    identical.    identical.    identical.    identical.   
Converged.

done
dvtditr (aa) Version 7.310 alg=A, model=BLOSUM62, 1.53, -0.00, -0.00, noshift, amax=0.0
0 thread(s)


Strategy:
 L-INS-i (Probably most accurate, very slow)
 Iterative refinem

In [30]:
!time kalign seqs/protein.fa seqs/kalign_protein.fa

EXTRA :2

Kalign version 2.04, Copyright (C) 2004, 2005, 2006 Timo Lassmann

        Kalign is free software. You can redistribute it and/or modify
        it under the terms of the GNU General Public License as
        published by the Free Software Foundation.

reading from STDIN: found no sequences.

reading from seqs/protein.fa: found 10 sequences
reading from seqs/kalign_protein.fa: found 10 sequences
Aligning 20 protein sequences with these parameters:
        54.94940948	gap open penalty
        8.52492046	gap extension
        4.42409992	terminal gap penalty
        0.20000000	bonus
Alignment will be written to stdout.

Distance Calculation:
     100 percent done
Alignment:
     100 percent done
>SUP35_Kla_AB039749_1
MSDQQN-QDQGQGQGYNQYNQYGQ--------YNQYYNQQGYQGYNGQQGAPQG--YQAY
QAYGQQPQGAYQGYNP--QQAQGYQPYQGYNAQQQG-YNAQQGGHNNNYNKNYNNKNSYN
NYNKQGYQGAQGYNAQQPTGYAAPAQSSSQGMTLKDFQNQQ-GSTNAAKPKPKLKLASSS
GIKLVGAKKPVAPKTEKTDESKEATKTT----DDNEEAQSELPKIDDLKISEAEKPKTKE
NTPSADDTSSEKTTSAKADTS

In [31]:
!time prank -d=seqs/protein.fa -o=seqs/prank_protein.fa


-----------------
 PRANK v.170427:
-----------------

Input for the analysis
 - aligning sequences in 'seqs/protein.fa'
 - using inferred alignment guide tree
 - option '+F' is not used; it can be enabled with '+F'
 - external tools available:
    MAFFT for initial alignment

Correcting (arbitrarily) for multifurcating nodes.
Correcting (arbitrarily) for multifurcating nodes.

Generating multiple alignment: iteration 1.

Alignment score: 1555
Correcting (arbitrarily) for multifurcating nodes.

Generating multiple alignment: iteration 2.

Alignment score: 1522
Correcting (arbitrarily) for multifurcating nodes.

Generating multiple alignment: iteration 3.

Alignment score: 1377
Correcting (arbitrarily) for multifurcating nodes.

Generating multiple alignment: iteration 4.

Alignment score: 1377
Correcting (arbitrarily) for multifurcating nodes.

Generating multiple alignment: iteration 5.

Alignment score: 1377


Writing
 - alignment to 'seqs/prank_protein.fa.best.fas'

Analysis done. T

Сравним:
* clustalw
    * Время: 0.58 сек
    * New number of positions in clustal_protein.fasta-gb:  576  (80% of the original 719 positions)
    * т.е. скор 80%
* muscle
    * Время: 0.25 сек
    * New number of positions in muscle_protein.fa-gb:  553  (74% of the original 743 positions)
    * т.е. скор 74%
* mafft
    * Время: 0.56 сек
    * New number of positions in mafft_protein.fa-gb:  564  (74% of the original 759 positions)
    * т.е. скор 74%
* kalign
    * Время: 0.11 сек
    * New number of positions in kalign_protein.fa-gb:  574  (79% of the original 720 positions)
    * т.е. скор 79%
* prank
    * Время: 141.63 сек (но он крутой, знает филогенетику)
    * New number of positions in prank_protein.fa.best.fas-gb:  508  (65% of the original 780 positions)
    * т.е скор 65%
    
1. Выравнивания белков ниже скор дали, чем ДНК, это и логично, букв то больше (хотя есть похожие и нет)
2. Опять умный пранк дал скор ниже всех. Наверное, это вызвано тем, что сами последовательности действительно хуже выравниваются в жизни.

The end!